In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [ ]:
# third party
# run email server
from helpers import EmailServer
from helpers import SMTPTestServer

email_server = EmailServer()
smtp_server = SMTPTestServer(email_server)

In [ ]:
# syft absolute
import syft as sy
from syft import test_settings
from syft.service.job.job_stash import Job

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
request = [
    r for r in high_client.requests if r.code.service_func_name == "large_sample"
][0]

In [ ]:
request

In [ ]:
job = request.code(blocking=False)

In [ ]:
job

In [ ]:
assert isinstance(job, Job)

In [ ]:
query_limit_size = test_settings.get("query_limit_size", default=10000)

In [ ]:
result = job.wait()

assert len(result) == query_limit_size

In [ ]:
users_emails = email_server.get_emails_for_user(request.requesting_user_email)
number_of_emails = len(users_emails)

In [ ]:
# manually deposit
job_info = job.info(result=True)
response = request.deposit_result(job_info, approve=True)
assert isinstance(response, Job)

In [ ]:
users_emails = email_server.get_emails_for_user(request.requesting_user_email)
after_number_of_emails = len(users_emails)
assert after_number_of_emails > number_of_emails
# assert after_number_of_emails == number_of_emails + 1 # we should only send 1 email

In [ ]:
request = [
    r for r in high_client.requests if r.code.service_func_name.startswith("popular")
][0]

In [ ]:
request

In [ ]:
request.code

In [ ]:
assert "pending" in str(request.code.status).lower()

In [ ]:
assert "popular" in request.code.service_func_name

In [ ]:
# drink a can of approve classic 🥤
request.approve()

In [ ]:
users_emails = email_server.get_emails_for_user(request.requesting_user_email)
assert len(users_emails) > after_number_of_emails
assert len(users_emails) == after_number_of_emails + 1

In [ ]:
# job_info = job.info(result=True)
# # response = request.deposit_result(job_info, approve=True)
# # assert isinstance(response, Job)

In [ ]:
smtp_server.stop()

In [ ]:
server.land()